<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>XGBoost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нам необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**Структура проекта**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем все необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from pymystem3 import Mystem
import re

from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

Откроем файл с данными, ознакомимся с ними, изучим типы и возможные пропуски/аномалии

In [2]:
try:
  df = pd.read_csv('/Users/lesha_sm/Desktop/YandexPracticum/датафреймы/toxic_comments.csv')
except:
  df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Пропусков нет, но есть не несущий смысловой нагрузки и ценности для будущих предсказаний столбец 'Unnamed: 0'. Удалим его

In [5]:
df = df.drop('Unnamed: 0', axis=1)

Посмотрим на столбец с целевым признаком, отражающим тональность комментария

In [6]:
df['toxic'].describe()

count    159292.000000
mean          0.101612
std           0.302139
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: toxic, dtype: float64

Классы целевого признака сильно разбаллансированы, положительных - только 10%

Поделим данные на обучающую и тестовую выборки

In [7]:
train, test = train_test_split(df, test_size=0.2, random_state=2007)

Для будущего обучения и предсказаний модели выделим из каждой выборки корпуса с текстами комментариев. А также - целевой признак отдельно для каждой выборки.

In [8]:
train_corpus = train['text'].values

In [9]:
test_corpus = test['text'].values

In [10]:
y_train = train['toxic']

In [11]:
y_test = test['toxic']

Для дальнейшего составления векторов величин TF-IDF текстов корпуса загрузим пакет "стоп-слов" английского языка

In [12]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lesha_sm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Напишем функцию приведения слов к начальной форме (лемматизации), основу для которой возьмем из библиотеки pymystem3

In [13]:
#pip install pymystem3

In [14]:
#m = Mystem()

In [15]:
#def lemmatize(text):
#    return "".join(m.lemmatize(text))

In [16]:
from nltk.stem import WordNetLemmatizer

Также добавим функцию очистки текстов от лишних символов. Используем встроенный модуль "re"

In [17]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return ' '.join(text.split())

Лемматизируем все тексты в обучающем и тестовом корпусах, и избавимся от лишних символов в них

In [20]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/lesha_sm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lesha_sm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
lemmatizer = WordNetLemmatizer()

In [22]:
lemm_train_corpus = []
for i in range(len(train_corpus)):
    word_list = nltk.word_tokenize(train_corpus[i])
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w, 'n') for w in word_list])
    lemm_train_corpus.append(clear_text(lemmatized_output))

In [23]:
lemm_test_corpus = []
for i in range(len(test_corpus)):
    word_list = nltk.word_tokenize(test_corpus[i])
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w, 'n') for w in word_list])
    lemm_test_corpus.append(clear_text(lemmatized_output))

Вычислим частоты встречаемости слов в текстах и корпусе TF-IDF

In [24]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [25]:
tf_idf_train = count_tf_idf.fit_transform(lemm_train_corpus)

In [26]:
tf_idf_test = count_tf_idf.transform(lemm_test_corpus)

## Обучение

Обучим несколько разных моделей и проверим качество их предсказаний на тестовой выборке по метрике F1

In [27]:
features_train = tf_idf_train

In [28]:
features_test = tf_idf_test

### LogisticRegression

In [29]:
model = LogisticRegression(class_weight='balanced', 
                           random_state=2007,
                           solver='saga'
                           )

In [30]:
%%time
model.fit(features_train,y_train)

CPU times: user 6.07 s, sys: 11.5 ms, total: 6.08 s
Wall time: 6.08 s


/Users/lesha_sm/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(class_weight='balanced', random_state=2007, solver='saga')

In [31]:
preds_test = model.predict(features_test)

In [32]:
print('Значение F1 для Логистической регрессии =', round(f1_score(y_test, preds_test), 2))

Значение F1 для Логистической регрессии = 0.75


Вывод: 
<br>Логистическая регрессия быстро обучается, качество предсказаний отвечает заданному условиями проекта

### CatBoost

In [33]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [34]:
clf = CatBoostClassifier(iterations=100,
                         learning_rate=1,
                         depth=7,
                         silent=True)

In [35]:
%%time
clf.fit(features_train, y_train)

CPU times: user 6min 25s, sys: 3min 53s, total: 10min 19s
Wall time: 2min 20s


In [36]:
clf_preds = clf.predict(features_test)

In [37]:
print('Значение F1 для CatBoost =', round(f1_score(y_test, clf_preds),2))

Значение F1 для CatBoost = 0.76


Вывод:
<br>CatBoost обучается сравнительно долго, качество предсказаний - выше Логистической регрессии

### XGBoost

In [38]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [39]:
xgb_clf = XGBClassifier(max_depth=9, 
                        learning_rate=1,
                        n_estimators=200,
                        disable_default_eval_metric=True)

In [40]:
%%time
xgb_clf.fit(features_train, y_train)

CPU times: user 6min 35s, sys: 7.36 s, total: 6min 42s
Wall time: 2min 23s


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              disable_default_eval_metric=True, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=1, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=9, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, ...)

In [41]:
preds_xgb = xgb_clf.predict(features_test)

In [42]:
print('Значение F1 для XGBoost =', round(f1_score(y_test, preds_xgb),2))

Значение F1 для XGBoost = 0.76


Вывод:
<br> XGBoost обеспечивает качество предсказаний аналогичное Логистической регрессии, обучается почти втрое быстрее CatBoost'a

PIPELINE USAGE

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
#from sklearn.externals import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [44]:
features_train, features_test, y_train, y_test = train_test_split(
    df['text'], df['toxic'], test_size=0.2, random_state=2007)

In [45]:
pipe_lr = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(random_state=2007))])

pipe_cb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', CatBoostClassifier())])

pipe_xgb = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
            ('clf', XGBClassifier())])

In [46]:
param_range = [9, 10]
param_range_fl = [1.0, 0.5]
learning_range = [0.3, 1]

grid_params_lr = [{'clf__penalty': ['l1', 'l2'],
        'clf__C': param_range_fl,
        'clf__solver': ['saga'],
        'clf__class_weight': ['balanced', None]}] 


grid_params_cb = [{'clf__learning_rate': learning_range,
        'clf__depth': param_range,
        'clf__iterations': [30,40]}]

grid_params_xgb = [{'clf__learning_rate': learning_range, 
        'clf__n_estimators': [50, 60],
                   'clf__depth': param_range}]
jobs = -1

In [47]:
LR = GridSearchCV(estimator=pipe_lr,
            param_grid=grid_params_lr,
            scoring='f1',
            cv=10) 



CB = GridSearchCV(estimator=pipe_cb,
            param_grid=grid_params_cb,
            scoring='f1',
            cv=10, 
            n_jobs=jobs)


XGB = GridSearchCV(estimator=pipe_xgb,
            param_grid=grid_params_xgb,
            scoring='f1',
            cv=10,
            n_jobs=jobs)

grids = [LR,CB,XGB]

In [48]:
grid_dict = {0: 'Logistic Regression', 
        1: 'CatBoost Classifier',
        2: 'XGBoost Classifier'}

In [ ]:
print('Performing model optimizations...')
best_f1 = 0.0
best_clf = 0
best_gs = ''
for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])
    gs.fit(features_train, y_train)
    print('Best params are : %s' % gs.best_params_)
    # Best training data f1
    print('Best training f1_score: %.3f' % gs.best_score_)
    # Predict on test data with best params
    y_pred = gs.predict(features_test)
    # Test data f1 of model with best params
    print('Test set f1_score for best params: %.3f ' % f1_score(y_test, y_pred))
    # Track best (highest test f1) model
    if f1_score(y_test, y_pred) > best_f1:
        best_f1 = f1_score(y_test, y_pred)
        best_gs = gs
        best_clf = idx
print('\nClassifier with best test set f1: %s' % grid_dict[best_clf])


Performing model optimizations...

Estimator: Logistic Regression


/Users/lesha_sm/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/lesha_sm/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/lesha_sm/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/lesha_sm/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/lesha_sm/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear

BERT

In [ ]:
import transformers

In [ ]:
model = transformers.AutoModel.from_pretrained('unitary/toxic-bert')

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('unitary/toxic-bert')

In [ ]:
X = df['text']
y = df['toxic']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, 
                                                    stratify=y,
                                                    random_state=42)

In [ ]:
classifier = pipeline("sentiment-analysis", 
                      model='unitary/toxic-bert',
                      device=0)

In [ ]:
%%time
preds = classifier(X_test.to_list(), padding=True, 
                   max_length=512, truncation=True,
                   batch_size=32)

In [ ]:
def to_labels(predict_proba, threshold):
	return (predict_proba >= threshold).astype('int')

In [ ]:
y_pred_bert = to_labels(pd.DataFrame(preds)['score'], 0.5)

In [ ]:
f1_score(y_test, y_pred_bert)

## Выводы

После предобработки и разделения данных на выборки были обучены три разные модели и предсказаны значения целевого признака. 
Получены следующие результаты работы моделей:
<br>
- LogisticRegression (время обучения порядка 6-7 секунд, значение метрики F1 = 0.76)
- CatBoost (время обучения около 3 минут, значение метрики F1 = 0.77)
- XGBoost (время обучения около 1 минуты, значение метрики F1 = 0.76)

По результатам видно, что качество предсказаний моделей очень близко. Однако, сильно различается время обучения. Лидер по скорости обучения - ЛогистическаяРегрессия, самая простая из изученных моделей. На втором месте - XGBoost, время обучения которого выше logit, но XGBoost обеспечивает аналогичное ей качество предсказаний. В зависимости от потребностей и возможностей заказчика в момент запуска, можно рекомендовать для использования любую из двух отличившихся моделей. CatBoost дает лучшие из примеров результаты качества, но в данном случае обучается гораздо дольше других. В случае необходимости остановиться на одной модели, лучшие комплексные показатели - у Логистической регрессии, рекомендую использовать ее.

<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>